### 環境建置

In [116]:
# TEST necessary for when working with external scripts
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
import os
import pandas as pd
import numpy as np
import pickle

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneGroupOut, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from scipy.signal import butter, lfilter
from scipy import stats
from sklearn.preprocessing import MinMaxScaler  # 引入正規化方法


# 定義濾波器函數
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    db = np.sum(y**2)
    return db

def signal_frequency_band_energies(sampled_signal, frequency_bands, sampling_frequency, order=5):
    energies = []
    for bands in frequency_bands:
        energies.append(butter_bandpass_filter(sampled_signal, bands[0], bands[1], sampling_frequency, order))
    return energies

def extract_windows(data, window_size):
    """將信號分割成不重疊的窗口，每個窗口大小為window_size"""
    windows = []
    for i in range(0, len(data) - window_size, window_size):  # 每次跳過window_size的長度
        window = data[i:i + window_size]
        
        if len(window) == window_size:  # 確保每個窗口的大小是固定的
            windows.append(window)
    return np.array(windows)

### 資料讀取

In [151]:
BASE_PATH = "../WESAD/"

all_processed_windows = []

for i in range(2, 18):
    if i == 12: 
        continue

    subject_path = os.path.join(BASE_PATH, f'S{i}')
    pickle_file = os.path.join(subject_path, f'S{i}.pkl')

    with open(pickle_file, 'rb') as f:
        data = pickle.load(f, encoding='bytes')

        labels = data[b'label']
        signals = data[b'signal'][b'chest']

        acc_signal = signals[b'ACC'].flatten()
        ecg_signal = signals[b'ECG'].flatten()
        emg_signal = signals[b'EMG'].flatten()
        eda_signal = signals[b'EDA'].flatten()
        resp_signal = signals[b'Resp'].flatten()
        temp_signal = signals[b'Temp'].flatten()
        
        # 確保所有信號長度一致
        min_length = min(len(acc_signal),len(ecg_signal), len(emg_signal), len(eda_signal), len(resp_signal), len(temp_signal), len(labels))
        ecg_signal = ecg_signal[:min_length]
        emg_signal = emg_signal[:min_length]
        eda_signal = eda_signal[:min_length]
        resp_signal = resp_signal[:min_length]
        temp_signal = temp_signal[:min_length]
        labels = labels[:min_length]
        #print(len(acc_signal),len(ecg_signal), len(emg_signal), len(eda_signal), len(resp_signal), len(temp_signal), len(labels))

        window_size = 700
        
        # 獲取窗口的索引範圍
        windows_indices = [(i, i+window_size) for i in range(0, min_length-window_size, window_size)]
        
        # 對每個窗口處理
        for start_idx, end_idx in windows_indices:
            # 提取該窗口的各信號
            acc_window = acc_signal[start_idx:end_idx]
            ecg_window = ecg_signal[start_idx:end_idx]
            emg_window = emg_signal[start_idx:end_idx]
            eda_window = eda_signal[start_idx:end_idx]
            resp_window = resp_signal[start_idx:end_idx]
            temp_window = temp_signal[start_idx:end_idx]
            
            # 獲取該窗口的主要標籤（可以使用眾數或其他方法）
            window_labels = labels[start_idx:end_idx]
            label_mode = stats.mode(window_labels, keepdims=True)[0][0]
            
            # 計算ECG的頻帶能量
            ecg_bands = signal_frequency_band_energies(ecg_window, 
                                                      [[0.01, 0.04], [0.04, 0.15], [0.15, 0.4], [0.4, 1.0]], 
                                                      700)
            # # 對其他信號計算統計特徵（如平均值）
            acc_feature = np.mean(acc_window)
            ecg_feature = np.mean(ecg_window)
            emg_feature = np.mean(emg_window)
            eda_feature = np.mean(eda_window)
            resp_feature = np.mean(resp_window)
            temp_feature = np.mean(temp_window)
            
            # 將所有特徵和標籤組合成
            #window_features = np.concatenate([ecg_bands, [acc_feature,ecg_feature,emg_feature, eda_feature, resp_feature, temp_feature, label_mode]])
            window_features = np.concatenate([ecg_bands, 
                                acc_window, emg_window, 
                                eda_window, resp_window, temp_window, 
                                [label_mode]])
            print(window_features.shape)

            # 添加受試者ID作為特徵
            window_features = np.append(window_features, i)
            
            all_processed_windows.append(window_features)

columns = ['ECG_ULF', 'ECG_LF', 'ECG_HF', 'ECG_UHF','ACC',
           'EMG', 'EDA', 'Resp', 'Temp', 'Label', 'Subject']
processed_data_df = pd.DataFrame(all_processed_windows, columns=columns)

# 移除正規化部分，直接使用原始數據
processed_data_df['Label'] = processed_data_df['Label']
processed_data_df['Subject'] = processed_data_df['Subject']

# 保存為CSV
output_file = "../output/n_lab2_data.csv"
processed_data_df.to_csv(output_file, index=False)


(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)
(3505,)


ValueError: 11 columns passed, passed data had 3506 columns

In [144]:
df = pd.read_csv("../output/n_lab2_data.csv")

# 分開特徵和標籤

X = df.drop(columns=['Label', 'Subject'])  # 移除標籤 (y) 和 Subject
y = df['Label'] 



# 切分訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 模型

決策樹

In [134]:
# 初始化並訓練決策樹
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
model_name = "DecisionTree"

KNN

In [136]:

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)
model_name = "KNN"


隨機森林

In [138]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
model_name = "RandomForest"



AdaBoost 決策樹

In [140]:

model = AdaBoostClassifier(estimator=DecisionTreeClassifier(), n_estimators=50, random_state=42)
model.fit(X_train, y_train)
model_name = "Adaboost Decision Tree"

/Users/sean/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


線性判別分析 (LDA)


In [145]:
model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)
model_name = "LDA"

### 生成結果


In [146]:
# 預測測試集
print(model_name)
y_pred = model.predict(X_test)

# 計算準確率
accuracy = accuracy_score(y_test, y_pred)
print(f"模型準確率: {accuracy:.4f}")

# 輸出分類報告
print("分類報告：")
print(classification_report(y_test, y_pred, digits=4))


LDA
模型準確率: 0.4385
分類報告：
              precision    recall  f1-score   support

         0.0     0.4517    0.9193    0.6057      7860
         1.0     0.2496    0.0822    0.1237      3576
         2.0     0.5052    0.0491    0.0895      1995
         3.0     0.0000    0.0000    0.0000      1077
         4.0     0.0000    0.0000    0.0000      2381
         5.0     0.0000    0.0000    0.0000       160
         6.0     0.0000    0.0000    0.0000       155
         7.0     0.0000    0.0000    0.0000       167

    accuracy                         0.4385     17371
   macro avg     0.1508    0.1313    0.1024     17371
weighted avg     0.3138    0.4385    0.3098     17371



/Users/sean/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sean/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sean/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [62]:
logo = LeaveOneGroupOut()
y_pred = cross_val_predict(model, X, y, cv=logo, groups=df['Subject'])
accuracy = accuracy_score(y, y_pred)
print(f"模型準確率: {accuracy:.4f}")
report = classification_report(y, y_pred, digits=4)
print("分類報告：")
print(report)

模型準確率: 0.3448
分類報告：
              precision    recall  f1-score   support

         0.0     0.4356    0.6126    0.5092     39492
         1.0     0.2344    0.1960    0.2134     17611
         2.0     0.1561    0.1636    0.1597      9966
         3.0     0.0476    0.0124    0.0196      5575
         4.0     0.2683    0.0507    0.0852     11806
         5.0     0.0000    0.0000    0.0000       789
         6.0     0.0070    0.0025    0.0037       790
         7.0     0.0009    0.0012    0.0010       824

    accuracy                         0.3448     86853
   macro avg     0.1437    0.1299    0.1240     86853
weighted avg     0.3031    0.3448    0.3060     86853

